In [67]:
import os
import pandas as pd
import gensim
import nltk
import tempfile

from gensim.utils import simple_preprocess 
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
import pickle
from gensim.models import LdaMulticore
from bs4 import BeautifulSoup
import requests
import pprint
stemmer = PorterStemmer()

In [25]:
home = os.path.expanduser('~')
repo_path = 'Documents/repos/mystuff/lda101' #change to path of repo
repo = os.path.join(home, repo_path) 
pickled_data = os.path.join(repo, 'pickled_data')
models = os.path.join(repo,'models')

In [42]:
postfix = 'abcdefghijklmno'
def _assemble_w2v_file(postfix):
    processed_docs = tempfile.NamedTemporaryFile(mode='ab', delete=True)

    for letter in postfix:
        fname = os.path.join(pickled_data,'processed_docs.pkl.a'+letter)
        with open(fname, 'rb') as infile:
            processed_docs.write(infile.read())

    return processed_docs

def preprocess(text):
    result = []
    # convert document into list of lowercase tokens, filter based on token length
    for token in simple_preprocess(text):
        if token not in STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result
    

In [62]:
with _assemble_w2v_file(postfix) as processed_docs_file:
    f = open(processed_docs_file.name,'rb')
    processed_docs = pickle.load(f)
    f.close()
# load models and data
with open(os.path.join(repo,'dictionary.pkl'), 'rb') as file:
    dictionary = pickle.load(file)
# with open(os.path.join(repo,'bow_corpus.pkl'), 'rb') as file:
#     bow_corpus = pickle.load(file)
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs] # bow rep for each article
lda_model = LdaMulticore.load(os.path.join(models, 'lda.model'))


,article
0,dentary agriculture instead a part of delibera...
1,yuyao is county level city in the northeast of...
2,liu huaqing october january wa general of the ...
3,hui pan nationalism refers to the common ident...
4,man carrying shovel shovel is tool for digging...


In [63]:
class LDAModelDriver:
    def __init__(self, dictionary, bow_corpus, lda_model):
        self.dictionary = dictionary
        self.bow_corpus = bow_corpus
        self.lda_model = lda_model
    
    def lemmatize_stemming(self, text):
        '''
        lemmatize text, without pos tag, lemmatizer treats every word as noun. pos='v' tells lemmatizer to treat 
        each word as verb.
        '''
        word = WordNetLemmatizer().lemmatize(text, pos='v')
        return stemmer.stem(word)
    
    def preprocess(self, text):
        result = []
        # convert document into list of lowercase tokens, filter based on token length
        for token in simple_preprocess(text):
            if token not in STOPWORDS and len(token) > 3:
                result.append(self.lemmatize_stemming(token))
        return result

    def model_topics(self, text):
        bow_vector = self.dictionary.doc2bow(self.preprocess(text))
        for index, score in sorted(self.lda_model[bow_vector], key=lambda tup: -1*tup[1]):
            print("Score: {}\t Topic: {}".format(score, self.lda_model.print_topic(index, 5)))

In [68]:
from nytimes_scraper import get_text
lda_model_driver = LDAModelDriver(dictionary, bow_corpus, lda_model)
url = 'https://www.nytimes.com/1998/05/12/world/india-sets-3-nuclear-blasts-defying-a-worldwide-ban-tests-bring-a-sharp-outcry.html'
lda_model_driver.model_topics(get_text(url))

Score: 0.3781326413154602	 Topic: 0.009*"india" + 0.009*"countri" + 0.008*"relat" + 0.006*"embassi" + 0.005*"govern"
Score: 0.3524431586265564	 Topic: 0.007*"nuclear" + 0.004*"number" + 0.004*"nation" + 0.004*"text" + 0.003*"counti"
Score: 0.16132183372974396	 Topic: 0.011*"parti" + 0.008*"govern" + 0.008*"elect" + 0.006*"polit" + 0.005*"minist"
Score: 0.07931715250015259	 Topic: 0.005*"test" + 0.004*"club" + 0.003*"cricket" + 0.003*"play" + 0.003*"australia"
Score: 0.02503322809934616	 Topic: 0.010*"aircraft" + 0.008*"forc" + 0.007*"ship" + 0.006*"oper" + 0.004*"navi"
